In [ ]:
import pandas as pd
import numpy as np
# import plo
import plotly.express as px
import plotly.io as pio
pio.renderers.default='iframe'

DAU

In [ ]:
df = pd.read_csv('./downloads/2021-04 (2).csv')
df['cntr'] = np.where(df['country']=='RU', 'RU', None)
df['cntr'] = np.where(df['country']=='UA', 'UA', df['cntr'])
df['cntr'] = np.where(df['country']=='US', 'US', df['cntr'])
df['cntr'] = np.where((df['country']!='RU')&(df['country']!='UA')&(df['country']!='US'),'Other', df['cntr'])
df=df.drop(columns=['country'])
df=df.rename(columns={'cntr':'country'})
df


In [ ]:
dau = df.groupby(['event_date','country']).agg(dau=('user_id','nunique')).reset_index()

In [ ]:
dau

In [ ]:
dau.groupby(['country']).agg({'dau':'mean'}).sort_values(['dau'],ascending=False)

Retention

In [ ]:
df = pd.read_csv('./downloads/2021-04 (2).csv')
df['cntr'] = np.where(df['country']=='RU', 'RU', None)
df['cntr'] = np.where(df['country']=='UA', 'UA', df['cntr'])
df['cntr'] = np.where(df['country']=='US', 'US', df['cntr'])
df['cntr'] = np.where((df['country']!='RU')&(df['country']!='UA')&(df['country']!='US'),'Other', df['cntr'])
df=df.drop(columns=['country'])
df=df.rename(columns={'cntr':'country'})


In [ ]:
df['event_time'] = pd.to_datetime(df['event_time'])

In [ ]:
install = df[df['event'] == 'install']

In [ ]:
install = install.groupby(['user_id'], as_index=False).agg({'event_time':'min'})

In [ ]:
df = pd.merge(df, install, on=['user_id'], how='inner', suffixes=['','_install'])

In [ ]:
df['lifeday'] = (df['event_time'].dt.date - df['event_time_install'].dt.date).dt.days

In [ ]:
df = df[df['lifeday'] >= 0]

In [ ]:
ret = df.groupby(['lifeday','country']).agg({'user_id':'nunique'}).reset_index()

In [ ]:
ret0 = ret[ret['lifeday'] == 0]

In [ ]:
ret = pd.merge(ret, ret0, on=['country'], suffixes=['','_total'])

In [ ]:
ret = ret.drop(columns=['lifeday_total'])

In [ ]:
ret['ret'] = ret['user_id']/ret['user_id_total']
ret['text'] = (round(ret['ret']*100, 2)).astype(str) + '%'

In [ ]:
fig = px.line(ret, x='lifeday',y='ret', color='country')
fig.update_yaxes(dict(tickformat=".00%"))

ARPU

In [ ]:
df = pd.read_csv('./downloads/2021-04 (2).csv')
df['cntr'] = np.where(df['country']=='RU', 'RU', None)
df['cntr'] = np.where(df['country']=='UA', 'UA', df['cntr'])
df['cntr'] = np.where(df['country']=='US', 'US', df['cntr'])
df['cntr'] = np.where((df['country']!='RU')&(df['country']!='UA')&(df['country']!='US'),'Other', df['cntr'])
df=df.drop(columns=['country'])
df=df.rename(columns={'cntr':'country'})


In [ ]:
user_list = df[df['event'] == 'install']['user_id'].unique()
df = df[df['user_id'].isin(user_list)]

In [ ]:
arpu = df.groupby(['country']).agg({'user_id':'nunique','revenue_usd':'sum'}).reset_index()

In [ ]:
arpu['arpu'] = arpu['revenue_usd']/arpu['user_id']
arpu

ARPPU

In [ ]:
df = pd.read_csv('./downloads/2021-04 (2).csv')
df['cntr'] = np.where(df['country']=='RU', 'RU', None)
df['cntr'] = np.where(df['country']=='UA', 'UA', df['cntr'])
df['cntr'] = np.where(df['country']=='US', 'US', df['cntr'])
df['cntr'] = np.where((df['country']!='RU')&(df['country']!='UA')&(df['country']!='US'),'Other', df['cntr'])
df=df.drop(columns=['country'])
df=df.rename(columns={'cntr':'country'})

In [ ]:
user_list = df[df['event'] == 'install']['user_id'].unique()
df = df[df['user_id'].isin(user_list)]

In [ ]:
arppu = df[df['event'] == 'purchase'].groupby(['country']).agg({'user_id':'nunique','revenue_usd':'sum'}).reset_index()

In [ ]:
arppu['arppu'] = arppu['revenue_usd']/arppu['user_id']
arppu

LEVEL Funnel

In [ ]:
df = pd.read_csv('./downloads/2021-04 (2).csv')
df['cntr'] = np.where(df['country']=='RU', 'RU', None)
df['cntr'] = np.where(df['country']=='UA', 'UA', df['cntr'])
df['cntr'] = np.where(df['country']=='US', 'US', df['cntr'])
df['cntr'] = np.where((df['country']!='RU')&(df['country']!='UA')&(df['country']!='US'),'Other', df['cntr'])
df=df.drop(columns=['country'])
df=df.rename(columns={'cntr':'country'})

In [ ]:
df = df[df['event'].str.contains('level')]

In [ ]:
user_list = df[df['event'] == 'level_4']['user_id'].unique()
df = df[df['user_id'].isin(user_list)]

In [ ]:
df['event'] = df['event'].str.replace('level_','').astype(int)
df = df.rename(columns={'event':'level'})

In [ ]:
user_level = df.groupby(['level','country']).agg({'user_id':'nunique'}).reset_index()

In [ ]:
total_users = df.groupby(['country']).agg({'user_id':'nunique'}).reset_index()

In [ ]:
result = pd.merge(user_level, total_users, on='country', suffixes=['','_total'])

In [ ]:
result['conversion'] = result['user_id']/result['user_id_total']

In [ ]:
px.line(result, x='level', y='conversion',color='country')

In [ ]:
px.bar(result, x='level', y='conversion',color='country',barmode='group')